In [1]:
import numpy as np
import scipy.stats as sts
import scipy.optimize as opt

data = np.loadtxt('NewMacroSeries.txt', delimiter = ',')
datac_t, datak_t, dataw_t, datar_t, datay_t = data[:,0], data[:,1], data[:,2], data[:,3], data[:,4]

In [23]:
S=1000
T=100
#draw from uniform distribution
sim = np.random.uniform(0,1,(T,S))
#define k_1
k_1 = datak_t.mean()


In [37]:
#test simulations
sim

array([[ 0.04414   ,  0.64147177,  0.8289618 , ...,  0.17537956,
         0.4322922 ,  0.60759999],
       [ 0.22099387,  0.6386433 ,  0.04623035, ...,  0.38034005,
         0.43632197,  0.4982348 ],
       [ 0.43581431,  0.77365043,  0.50478992, ...,  0.59897687,
         0.33040179,  0.22661248],
       ..., 
       [ 0.5422574 ,  0.86160752,  0.12707669, ...,  0.82116861,
         0.01312334,  0.57358739],
       [ 0.9118383 ,  0.26115295,  0.53163698, ...,  0.59561422,
         0.86994765,  0.99793065],
       [ 0.50318716,  0.96868295,  0.19921973, ...,  0.75472244,
         0.74633106,  0.25992767]])

In [48]:
#first, we define sumulated variables
def simvar(alpha, beta, rho, mu, sigma, sim):
    eps = sts.norm.ppf(sim,0,sigma)
    z = np.zeros((T,S))
    z_t_1 = mu
    for i in range(T):
        z_t = rho*z_t_1 + (1-rho)*mu + eps[i,:]
        z[i,:] = z_t
        z_t_1 = z_t
    
    k = np.zeros((T,S))
    k_lag = np.zeros((T,S))
    k_t_1 = k_1
    for i in range(T):
        k_t = alpha*beta*np.exp(z[i,:])*k_t_1**alpha
        k_lag[i,:] = k_t_1
        k[i,:] = k_t
        k_t_1 = k_t

    w = (1-alpha)*np.exp(z)*k_lag**alpha
    r = alpha*np.exp(z)*k_lag**(alpha-1)
    c = w + r*k_lag-k
    y = np.exp(z)*k_lag**alpha
    
    return c,k_lag,k,w,r,y

In [49]:
simvar(0.5,0.99,0.5,8,0.5, sim)

(array([[ 1654722.22826468,  4651090.12167758,  6239743.77724981, ...,
          2433534.94639375,  3563107.86557771,  4447928.35108375],
        [  852406.54052816,  4202226.9488852 ,  2035529.09518963, ...,
          1581140.79875041,  2488246.52198493,  3357919.89525006],
        [  846308.52928529,  5085149.08517478,  1581775.88817248, ...,
          1751870.4174531 ,  1775428.39168725,  1940045.00889396],
        ..., 
        [ 2113918.81679386,  1962157.46828364,   380335.79811198, ...,
          3354961.20113947,   878149.21373268,  1997254.10750135],
        [ 4125131.67004532,  1706882.75203914,   571570.81833844, ...,
          4054782.39029105,  1545348.75967032,  9057063.16875307],
        [ 4193268.0614769 ,  4466349.276999  ,   582530.50477763, ...,
          5163031.09227501,  2714892.51410737,  6741895.72372484]]),
 array([[ 6643985.13829907,  6643985.13829907,  6643985.13829907, ...,
          6643985.13829907,  6643985.13829907,  6643985.13829907],
        [ 1621955.

In [60]:
#Let's define data moment and model moment

def sim_mom(alpha, beta, rho, mu, sigma, sim):
    c,k_lag,k,w,r,y = simvar(alpha, beta, rho, mu, sigma, sim)
    mmt1 = np.mean(c)
    mmt2 = np.mean(k)
    mmt3 = np.mean(c/y)
    mmt4 = np.mean(np.std(y,axis=0))
    corr_c = np.zeros((1,S))
    corr_ck = np.zeros((1,S))
    
    for i in range(S):
        corr_c[0,i] = np.corrcoef(c[0:99,i],c[1:100,i])[0,1]
        corr_ck[0,i] = np.corrcoef(c[:,i],k_lag[:,i])[0,1]
    
    mmt5 = corr_c.mean()
    mmt6 = corr_ck.mean()
    sim_mom = np.array([mmt1,mmt2,mmt3,mmt4,mmt5,mmt6])
    return sim_mom
#test
print(sim_mom(.5, .99, .5, 10, .5,sim))

mean_ct = datac_t.mean()
mean_kt = datak_t.mean()
mean_ctyt = (datac_t/datay_t).mean()
var_yt = datay_t.var()
corr_ct= np.corrcoef(datac_t[0:99],datac_t[1:100])[0,1]
corr_ctkt= np.corrcoef(datac_t,datak_t)[0,1]
data_mom=np.array([mean_ct, mean_kt,mean_ctyt,var_yt,corr_ct,corr_ctkt])
#test
print(data_mom)


def err_vec(alpha, beta, rho, mu, sigma, sim):
    model_moms = sim_mom(alpha, beta, rho, mu, sigma, sim)
    return (model_moms - data_mom)/data_mom

def SMM_crit(params, *args):
    alpha, rho, mu, sigma = params
    beta, sim = args
    err = err_vec(alpha, beta, rho, mu, sigma, sim)
    W = np.eye(6)
    return err @ W @ err.T



[  1.71354384e+08   1.67961228e+08   5.05000000e-01   3.29839167e+08
   7.10278028e-01   7.12968261e-01]
[  9.28179049e+06   6.64398514e+06   5.84200000e-01   2.83778251e+13
   9.40559181e-01   9.40803054e-01]


In [62]:
#SMM simulation
beta=0.99
params_init = np.array([0.5, 0.9, 9, 0.2])
bounds = ((1e-10,1-1e-10),(-1+1e-10,1-1e-10),(5,14),(1e-10,1.1))
result = opt.minimize(SMM_crit, params_init, args = (beta,sim), method = 'L-BFGS-B', bounds = bounds)
alpha_smm, rho_smm, mu_smm, sigma_smm = result.x
print(result)

print('alpha SMM = ', alpha_smm)
print('rho SMM = ', rho_smm)
print('mu SMM = ', mu_smm)
print('sigma SMM = ', sigma_smm)

      fun: 1.0000045053802726
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  2.81474843e-03,  -8.12683254e-06,   1.51656465e-04,
        -7.55795426e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 310
      nit: 33
   status: 0
  success: True
        x: array([  4.21103313e-01,   9.10131581e-01,   9.96870653e+00,
         3.43697937e-04])
alpha SMM =  0.421103312877
rho SMM =  0.910131580568
mu SMM =  9.96870652564
sigma SMM =  0.000343697937097
